In [1]:
#WARNING TRACK OPTION NEEDS SIGNIFICANT MEMORY
import sys
import json
import time
import os
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse.linalg import svds
import scipy.sparse
import matplotlib.pyplot as plt
import csv
import operator

In [2]:
data_location_desktop = "/home/ubutubiggerbetter/shared/data/spotify_data/data/new_new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/tmp_test"
raw_data_location = os.path.join(data_location_desktop, '..')
data_location = data_location_desktop
save_location = data_location


isdir = os.path.isdir(data_location)
if isdir == False:
    print(data_location, ' does not exist')


In [3]:
Usparse = scipy.sparse.load_npz(os.sep.join((data_location, 'U.npz')))
VTsparse = scipy.sparse.load_npz(os.sep.join((data_location, 'VT.npz')))
playlist_file = os.sep.join((data_location, "tr_playlistname_index_new.csv"))
full_char_mat = scipy.sparse.load_npz(os.sep.join((data_location, 'track_full_char_mat.npz')))
feature_file = os.sep.join((data_location, "track_index_new.csv"))

In [4]:
playlist_indices = sorted([38, 738, 807, 806, 1308, 1563, 1658, 1653, \
                   1965, 4278, 4470, 4553, 5947, 21944, 23637, \
                   29330, 371355, 14613, 377409, 377408, 782899, \
                   804775, 4074])

In [5]:
#See below for how it's run
def genrefier(playlist_indices, Usparse, VTsparse, playlist_file, track_file, latent_factor_count = 1, top_values = 10):
    start_time=  time.time()
    #Convert to useable array:
    
    U = Usparse.toarray()
    VT = VTsparse.toarray()
    #Flip around:
    n = U.shape[1]
    U[:,0:n] = U[:, n-1::-1]
    VT[0:n, :] = VT[n-1::-1,:]

    print('Prepared Information Arrays, elapsed time: ', time.time() - start_time)
    start_time = time.time()
    #Recover Track ingormation
    track_file = os.sep.join((data_location, "track_index_new.csv"))
    track_dict = {}
    track_index_dict = {}
    fobj = open(track_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        track_dict[row['track_name']] = row['index']
        track_index_dict[row['index']] = row['track_name']
    fobj.close()
    
    #Recover Playlist Information:
    playlist_dict= {}
    pl_index_dict = {}
    fobj = open(playlist_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        playlist_dict[row['playlist_name']] = row['index']
        pl_index_dict[row['index']] = row['playlist_name']
    fobj.close()
    print('Read in playlist info, elapsed time: ', time.time() - start_time)
    start_time = time.time()
    #Get songs that are in matrix:
    print('Songs in Playlist: ')
    for i in playlist_indices:
        name = track_index_dict[str(i)]
        print(name)

    #Turn playlist_indices into useable matrix:
    plist = csc_matrix((U.shape[0], 1))
    plist[playlist_indices]= 1 
    
    #Map into latent space
    genre_array = plist.transpose() * U
    #Get the top genres:
    top_genres = np.argsort(genre_array)
    top_genres = np.fliplr(top_genres)
    
    for i in range(0, latent_factor_count):
        top_genre = top_genres[0][i]
        #Get associated playlists from VT:
        associated_playlists = np.argsort(VT[top_genre,:])
        #flip so max is at the top
        associated_playlists = associated_playlists[::-1]
        
        print('='*50)
        print('TOP GENRE ', i)
        print('Best Described As (from Playlist Names):')
        for j in range(0, top_values):
            playlist_name = pl_index_dict[str(associated_playlists[j])]
            print(playlist_name, end=', ')
    print('\nDone. Time Elapsed: ', time.time() - start_time)

In [6]:
#Will read out the most representative genre, with a given playlist:
genrefier(playlist_indices, Usparse, VTsparse, playlist_file, feature_file, top_values = 8)

Prepared Information Arrays, elapsed time:  4.692011833190918
Read in playlist info, elapsed time:  10.484344482421875
Songs in Playlist: 
Mr. Brightside
Tubthumping
Semi-Charmed Life
Jumper - 1998 Edit
Little Lion Man
Roses
Wonderwall
Creep - Acoustic
Hey There Delilah
Welcome To The Jungle
Shoop (Re-Recorded) [Remastered]
Drops of Jupiter
Ophelia
Kryptonite
First Date
Drunken Lullabies
A Sky Full Of Stars - Radio Edit
Pumpin Blood - The Chainsmokers Remix
In My Youth
Rolling In the Deep (Acoustic Tribute to Adele)
Sweet Child O' mine (Acoustic)
I Gotta Feeling (Blackeyed Peas)
I'm Shipping Up to Boston


/home/ubutubiggerbetter/anaconda3/envs/w_pyspark/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


TOP GENRE  0
Best Described As (from Playlist Names):
Alt_420, alt/rock, Alternative_978, alt_192, Alternative/Pop, Alternative _49, Not Rap_11, rockish_19, 
Done. Time Elapsed:  0.9518022537231445
